# Projeto 2 - Ciência dos Dados

Nome: Gabriela Choichit Giosa

Nome: João Pedro Faria de Araujo

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [4]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import pandas as pd
import numpy as np
import re
from IPython.display import display
import string

In [5]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [12]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [15]:
#Produto escolhido:
produto = '#Joker'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [16]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 401

Salvando os dados em uma planilha Excel:

In [17]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

lendo o arquivo e fazendo a tabela de relativos e não relativos

In [6]:
joker = pd.read_excel('#JokerFull.xlsx')
joker_idx = joker.set_index('Treinamento')
joker_relevante = joker_idx[joker_idx.Classe == 1]
joker_nao_relevante = joker_idx[joker_idx.Classe == 0]

In [83]:
lista_palavras_relevantes = str(set(joker_relevante.index))
cleanup(lista_palavras_relevantes)
lista_palavras_relevantes = lista_palavras_relevantes.split()
lista_palavras_relevantes

['{"rt',
 '@pnemiroff:',
 '#joker',
 'is',
 'one',
 'of',
 'the',
 'most',
 'unnerving',
 'movies',
 "i've",
 'seen',
 'in',
 'years.',
 'joaquin',
 'phoenix',
 'is',
 'astounding',
 '-',
 'the',
 'physicality',
 'of',
 'his',
 'work',
 'i…",',
 "'rt",
 '@moviemantz:',
 'joker:',
 'not',
 'your',
 'daddy’s',
 '#joker',
 'movie.',
 'riveting',
 '&amp;',
 'engrossing,',
 'unnerving',
 '&amp;',
 'disturbing,',
 'intense',
 '&amp;',
 'very',
 'violent.',
 'a',
 'cross',
 "betwee…',",
 "'rt",
 '@patrckhenrque:',
 '"can',
 'you',
 'introduce',
 'me',
 'as',
 'the',
 '#joker',
 '?"',
 "https://t.co/46tnqtjjjt',",
 "'rt",
 '@victim815:',
 'i',
 'love',
 'joaquin',
 'phoenix’s',
 'character',
 'arc.',
 '#joker',
 "https://t.co/ekja3krupv',",
 "'who",
 'else',
 'is',
 'excited',
 'for',
 'this',
 '#joker',
 '#jokermovie',
 "https://t.co/hlwmkhuzy5',",
 "'@comicbook",
 'injustice',
 'fall.',
 "#joker',",
 "'rt",
 '@mikerimbaud1:',
 '@joncoopertweets',
 'didn’t',
 'he',
 'say',
 'he',
 'was',
 'on

In [86]:
lista_palavras_relevantes2 = str(set(joker_relevante.index))

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
lista_palavras_relevantes2 = lista_palavras_relevantes2.translate(translator).split()
lista_palavras_relevantes2

for k in range(len(lista_palavras_relevantes)):
      if lista_palavras_relevantes2[k] not in lista_palavras_relevantes:
        print(lista_palavras_relevantes2[k])

rt
pnemiroff
ve
years
i…
rt
moviemantz
amp
engrossing
amp
amp
betwee…
rt
patrckhenrque
https
t
co
46tnqtjjjt
rt
victim815
arc
https
t
co
ekja3krupv
jokermovie
https
t
co
hlwmkhuzy5
comicbook
fall
rt
mikerimbaud1
joncoopertweets
“joking
”
thejoker
trumpotheclown
lol
liarandchief
rt
berghain38
n
nign
nvariety
5
5
nhollywood
thr
5
5
nempire
magazine
ntotal
magazine
1…
https
t
co
hjpnjz1wwx
growing
yes
n
nright
sat
rt
nunosarnadas
jokermovie
talenthouse
posterspy
altmovieposters
printedinblood
posterposse
https
…
shooters
https
t
co
wwi18y3hyl
https
t
co
pmhcnmdiiv
tours
embarassment
atleast
n
n
modiinrussia
n
nhttps
t
co
cvzxrmf8b9
jokes
https
t
co
hi2ueb919v
rt
getfandom
87
https
t
co
kq7mnfcjv0
marvel
wbpictures
dccomics
nhttps
t
co
oc0fhba7w5
https
t
co
zhroiomtka
haven
t
jokermovie
dccomics
wbpictures
jokermovie
dcentertainment
wbpictures
https
t
co
zlbcvkryu4
untitled
tiff19
xa0
n
n40
free
impressions
…
https
t
co
f6fsxddc9z
rt
iamhamedabbasi
tiff19
https
t
co
baovd4hmiq
rt
usweekly


tsco0u97f1
n
n
cinehunch
https
t
co
3mheo2zby2
sucked
n
nhttps
t
co
oubb9hkqxp
n
n
itchaptertwo
pennywise
horrormovies
rt
dame
falcon
quinn
🙃
mortalkombat11
jo…
rt
tvsnmovies
joker2019
arkhamcomedy
jokermovienews
leopasaribu2
jokermovie0
kawaisugiruwa
jokermovie


In [12]:

#lista_1 = list(set(joker_relevante.index))
#z = len(list(set(joker_relevante.index)))
#lista_palavras_relevantes2 = []

#for i in range(z):
#    lista_palavras_relevantes2 = lista_palavras_relevantes2 + lista_1[i].split()
#print(len(lista_palavras_relevantes2))
#lista_palavras_relevantes2 = [str(i) for i in lista_palavras_relevantes2]
#lista_palavras_relevantes2 = [''.join(c for c in s if c not in string.punctuation) for s in lista_palavras_relevantes2]
#lista_palavras_relevantes2 = [s for s in lista_palavras_relevantes2 if s]
 


#for k in range(len(lista_palavras_relevantes)):
#      if lista_palavras_relevantes[k] not in lista_palavras_relevantes2:
#        print(lista_palavras_relevantes[k])

Frequencias relevante relativas e absolutas

In [73]:
serie_relevante = pd.Series(lista_palavras_relevantes)
tabela_relevante = serie_relevante.value_counts()
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa

#joker                                               0.035314
the                                                  0.033018
'rt                                                  0.026414
a                                                    0.021533
to                                                   0.015217
is                                                   0.014068
of                                                   0.012633
i                                                    0.010336
and                                                  0.009187
this                                                 0.008326
in                                                   0.007752
for                                                  0.007752
movie                                                0.006891
#jokermovie                                          0.006891
as                                                   0.006316
joaquin                                              0.006029
at      

lista com as palavras não relevantes filtradas

In [10]:
lista_palavras_nao_relevantes = str(set(joker_nao_relevante.index))
cleanup(lista_palavras_nao_relevantes)
lista_palavras_nao_relevantes = lista_palavras_nao_relevantes.split()
lista_palavras_nao_relevantes
lista_palavras_nao_relevantes

["{'rt",
 '@markosovrlic:',
 'when',
 'your',
 'basketball',
 'iq',
 'is',
 'high',
 'so',
 'you',
 'don’t',
 'need',
 'to',
 'be',
 'fit',
 '😂',
 '#nikolajokic',
 '#joker',
 '#serbia',
 '#nba',
 '#denver',
 '#basketball',
 "#world…',",
 "'@talkingspidey",
 'it’s',
 'a',
 'many',
 'way',
 'tie',
 'between',
 '#bane,',
 '#batman,',
 '#beast,',
 'captain',
 'america,',
 'captain',
 'cold,',
 '#deadshot,',
 '#deathstroke,',
 'dr.',
 'doom,',
 '#hulk,',
 '#joker,',
 'jonah',
 'hex,',
 'lex',
 'luthor,',
 '#loki,',
 '#magneto,',
 '#punisher,',
 'red',
 'sonja,',
 'she-hulk,',
 '#sinestro,',
 'spider-man,',
 '#thing,',
 '#thor,',
 '#venom,',
 '#wolverine,',
 'and',
 'many',
 "more',",
 "'rt",
 '@afis8:',
 'gonna',
 'have',
 'to',
 'sit',
 'with',
 '#joker',
 'for',
 'a',
 'little',
 'while,',
 'but',
 'i',
 'will',
 'say',
 'this',
 '-',
 'a',
 'lot',
 'of',
 'incels',
 'are',
 'going',
 'to',
 'feel',
 'pretty',
 'fucking',
 "empower…',",
 '"rt',
 '@nikkucremona:',
 'the',
 'popular',
 "'#l

Frequencias nao relevantes relativas e absolutas

In [11]:
serie_nao_relevante = pd.Series(lista_palavras_nao_relevantes)
tabela_nao_relevante = serie_nao_relevante.value_counts()
tabela_nao_relevante_relativa = serie_nao_relevante.value_counts(True)
tabela_nao_relevante_relativa

the                                                  0.029779
#joker                                               0.027767
'rt                                                  0.021328
a                                                    0.016097
to                                                   0.015292
of                                                   0.013682
is                                                   0.009256
for                                                  0.008853
and                                                  0.008853
on                                                   0.008048
in                                                   0.007243
you                                                  0.006841
this                                                 0.006439
"rt                                                  0.006036
i                                                    0.005634
joker                                                0.004024
#batman 

joker total

In [52]:
lista_palavras = []
z = len(joker_idx)
for i in range(z):
    lista_palavras = lista_palavras + joker_idx.index[i].split()

lista_palavras = [str(i) for i in lista_palavras]
lista_palavras = [''.join(c for c in s if c not in string.punctuation) for s in lista_palavras]
lista_palavras = [s for s in lista_palavras if s]
lista_palavras

['rt',
 'strraj07',
 'jokermovie',
 'poster',
 'art',
 'submission',
 'for',
 'talenthouse',
 'toddphillips7',
 'warnerbrosindia',
 'posterspy',
 'altmovieposters',
 'posterpo…',
 'rt',
 'imrkgupta',
 'these',
 'two',
 'guysawwweesomee🤡🎭🎊🎉🙌💕',
 'joker',
 'jokermovie',
 'october4',
 'jokerfilm',
 'joaquinphoenix',
 'heathledger',
 'httpstcogpaf…',
 'rt',
 'luizfernandoj',
 'another',
 'prestigious',
 'european',
 'film',
 'festival',
 'adds',
 'jokermovie',
 'to',
 'their',
 'lineup',
 'joker',
 'will',
 'be',
 'one',
 'of',
 'the',
 'gala',
 'premieres…',
 'stealthy',
 'android',
 'trojan',
 'spy',
 'signs',
 'you',
 'up',
 'for',
 'premium',
 'subscriptions',
 'via',
 'bleepincomputer',
 'proficio',
 'threatnews',
 'cybersecurity',
 'mssp',
 'mdr',
 'joker',
 'trojan',
 'httpstcomuyi7huwgn',
 'rt',
 'brooklynbatman',
 '2019',
 'is',
 'the',
 'year',
 'of',
 'the',
 'clown',
 'itchaptertwo',
 'joker',
 'httpstcohhbpx62nux',
 'rt',
 'comicbook',
 'a',
 'detailed',
 'plot',
 'synopsis',


Frequencias nao relevantes e relevantes relativas e absolutas

In [53]:
serie_total = pd.Series(lista_palavras)
tabela_total = serie_total.value_counts()
tabela_total_relativa = serie_total.value_counts(True)
tabela_total_relativa

joker                 0.051967
rt                    0.035957
the                   0.032836
a                     0.019403
of                    0.015061
is                    0.014111
to                    0.013569
i                     0.010583
this                  0.009227
jokermovie            0.009091
and                   0.008141
in                    0.007734
for                   0.007056
movie                 0.006377
joaquin               0.005834
one                   0.005699
on                    0.005427
you                   0.005427
it                    0.005292
be                    0.004885
that                  0.004342
batman                0.004071
at                    0.004071
with                  0.003799
dccomics              0.003528
as                    0.003528
2019                  0.003528
so                    0.003392
dc                    0.003392
but                   0.003392
                        ...   
httpstcokzujolgnrd    0.000136
maxhwell

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**